<a href="https://colab.research.google.com/github/doronin99/Arboretum/blob/api_v1/notebooks/lgo_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies Installing

In [ ]:
pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.3 MB/s eta 0:00:00


In [ ]:
import gdown
import os
import pickle
import zipfile

import numpy as np
import pandas as pd

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## Data importing

In [ ]:
# URL на zip-файл с датасетом
file_url = 'https://drive.google.com/uc?id=11Ch9oltgzsbKFn_F-VwZdTo0QkxMXamq'

# Загрузка zip-файла
output_path = 'data_original.zip'
gdown.download(file_url, output_path, quiet=False)

# Распаковка архива
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Удаление zip-файла
os.remove(output_path)

# Загрузка CSV-файла в Pandas DataFrame
csv_file_path = '/content/from_59_stat_202312010733.csv'
df = pd.read_csv(csv_file_path, delimiter=';')

# Вывод первых нескольких строк DataFrame
df.head()

Downloading...
From: https://drive.google.com/uc?id=11Ch9oltgzsbKFn_F-VwZdTo0QkxMXamq
To: /content/data_original.zip
100%|██████████| 102M/102M [00:01<00:00, 69.8MB/s] 


,period_id,user_id,qual,lgo
0,59,2408253,NONE,224.4
1,59,237267,S3,3452.3
2,59,2720895,NONE,5.9
3,59,2676227,S1,772.3
4,59,2839593,NONE,63.4


## EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24517231 entries, 0 to 24517230
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   period_id  int64  
 1   user_id    int64  
 2   qual       object 
 3   lgo        float64
dtypes: float64(1), int64(2), object(1)
memory usage: 748.2+ MB


## Preprocessing

In [ ]:
def filter_users(df, selected_qualifications, exclude_all_none=True):
    # Фильтруем по выбранным квалификациям: исключаем те user_id, которые имели за рассматриваемый период квалификации не соответствующие выбранным
    valid_user_ids = df.groupby('user_id')['qual'].apply(set).apply(lambda x: set(x).issubset(selected_qualifications)).reset_index()
    valid_user_ids = valid_user_ids[valid_user_ids['qual'] == True]['user_id']
    filtered_df = df[df['user_id'].isin(valid_user_ids)]

    # В зависимости от параметра exclude_all_none решаем исключать или не исключать пользователей с 'NONE' во всех периодах
    if exclude_all_none:
        valid_user_ids = filtered_df.groupby('user_id')['qual'].apply(lambda x: not all(qual == 'NONE' for qual in x)).reset_index()
        valid_user_ids = valid_user_ids[valid_user_ids['qual'] == True]['user_id']
        filtered_df = filtered_df[filtered_df['user_id'].isin(valid_user_ids)]

    return filtered_df

In [ ]:
%%time
selected_qualifications = ['NONE', 'S1', 'S2', 'S3', 'L']
filtered_df = filter_users(df, selected_qualifications, exclude_all_none=True)
filtered_df

CPU times: user 1min 29s, sys: 2.44 s, total: 1min 31s
Wall time: 1min 33s


,period_id,user_id,qual,lgo
1,59,237267,S3,3452.3
3,59,2676227,S1,772.3
10,59,2652981,S1,921.4
11,59,685520,S3,2565.2
13,59,2699376,NONE,213.5
...,...,...,...,...
24496792,83,3928499,S1,762.3
24498388,83,3930389,S1,772.1
24502863,83,3935456,S1,862.0
24506177,83,3939081,S2,1506.0


In [ ]:
filtered_df['prev_qual'] = filtered_df.groupby('user_id')['qual'].shift(1)
filtered_df['prev_lgo'] = filtered_df.groupby('user_id')['lgo'].shift(1)
filtered_df.dropna(inplace=True)
filtered_df

,period_id,user_id,qual,lgo,prev_qual,prev_lgo
615798,60,2805597,S2,1502.4,S1,1134.8
615799,60,2900850,S3,2505.6,S3,2810.3
615800,60,2078663,S1,760.5,NONE,558.2
615801,60,146571,L,5406.8,L,6726.0
615803,60,1340574,S1,752.7,S1,763.0
...,...,...,...,...,...,...
24481892,83,2246534,NONE,729.9,NONE,453.1
24481925,83,3681650,NONE,436.6,NONE,301.1
24481938,83,3753104,S1,1494.6,S2,2031.9
24481975,83,2413025,NONE,422.2,NONE,322.5


In [ ]:
def add_time_features(df, current_month, current_year):
    # Клонируем DataFrame, чтобы не изменять оригинальный
    processed_df = df.copy()

    # Находим последнее значение period_id
    last_period_id = processed_df['period_id'].max()

    # Создаем новые признаки: месяц, квартал и год
    processed_df['month'] = ((processed_df['period_id'] - last_period_id + current_month - 1) % 12) + 1
    processed_df['quarter'] = ((processed_df['month'] - 1) // 3) + 1
    processed_df['year'] = current_year + (processed_df['period_id'] - last_period_id + current_month - 1) // 12

    # Группировка по типу периода и вычисление среднего значения lgo
    average_lgo_by_month = processed_df.groupby('month')['lgo'].mean().reset_index()
    average_lgo_by_quarter = processed_df.groupby('quarter')['lgo'].mean().reset_index()
    average_lgo_by_year = processed_df.groupby('year')['lgo'].mean().reset_index()

    # Присоединение средних значений к исходному DataFrame
    processed_df = pd.merge(processed_df, average_lgo_by_month, on='month', how='left', suffixes=('', '_avg_month'))
    processed_df = pd.merge(processed_df, average_lgo_by_quarter, on='quarter', how='left', suffixes=('', '_avg_quarter'))
    processed_df = pd.merge(processed_df, average_lgo_by_year, on='year', how='left', suffixes=('', '_avg_year'))

    return processed_df

# Пример использования
current_month = 10
current_year = 2023
processed_data = add_time_features(filtered_df, current_month, current_year)
processed_data

,period_id,user_id,qual,lgo,prev_qual,prev_lgo,month,quarter,year,lgo_avg_month,lgo_avg_quarter,lgo_avg_year
0,60,2805597,S2,1502.4,S1,1134.8,11,4,2021,824.605001,746.237656,1025.515425
1,60,2900850,S3,2505.6,S3,2810.3,11,4,2021,824.605001,746.237656,1025.515425
2,60,2078663,S1,760.5,NONE,558.2,11,4,2021,824.605001,746.237656,1025.515425
3,60,146571,L,5406.8,L,6726.0,11,4,2021,824.605001,746.237656,1025.515425
4,60,1340574,S1,752.7,S1,763.0,11,4,2021,824.605001,746.237656,1025.515425
...,...,...,...,...,...,...,...,...,...,...,...,...
648889,83,2246534,NONE,729.9,NONE,453.1,10,4,2023,620.107170,746.237656,687.055354
648890,83,3681650,NONE,436.6,NONE,301.1,10,4,2023,620.107170,746.237656,687.055354
648891,83,3753104,S1,1494.6,S2,2031.9,10,4,2023,620.107170,746.237656,687.055354
648892,83,2413025,NONE,422.2,NONE,322.5,10,4,2023,620.107170,746.237656,687.055354


## Catboost

In [ ]:
# Загрузим данные
data = processed_data

# Разбивка данных на тренировочный и тестовый наборы
train_data = data[(data['period_id'] >= 59) & (data['period_id'] < 83)]
test_data = data[data['period_id'] == 83]

# Определение признаков и целевой переменной
features = ['user_id', 'prev_qual', 'prev_lgo', 'month', 'quarter', 'year']
cat_features = ['user_id', 'prev_qual']
target = 'lgo'

X_train, X_test = train_data[features], test_data[features]
y_train, y_test = train_data[target], test_data[target]

# Инициализация и обучение модели CatBoost
model = CatBoostRegressor(cat_features=cat_features, iterations=500, learning_rate=0.1, depth=6, l2_leaf_reg=3)
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=100)

# Прогнозирование на тестовых данных
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error on Test Data: {mse}')
print(f'R2: {r2}')

0:	learn: 910.6573748	test: 841.9538887	best: 841.9538887 (0)	total: 493ms	remaining: 4m 5s
100:	learn: 449.8018777	test: 418.7620836	best: 417.8854981 (59)	total: 36s	remaining: 2m 22s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 417.8854981
bestIteration = 59

Shrink model to first 60 iterations.
Mean Squared Error on Test Data: 174628.28896473616
R2: 0.7811329397514432


In [ ]:
with open('ml_model.pkl', 'wb') as file:
    pickle.dump(model, file)